# The Bird Reservoir

## Présentation du dataset

Il s'agit d'un ensemble de données d'enregistrements sonores d'oiseaux, un sous-ensemble spécifique recueilli à partir de la collection Xeno Canto pour former un ensemble de données équilibré couvrant 88 espèces couramment entendues au Royaume-Uni.

Lien vers les données : https://www.kaggle.com/datasets/rtatman/british-birdsong-dataset

In [11]:
%pip install librosa soundfile
import numpy as np
import librosa
import soundfile as sf


[notice] A new release of pip is available: 24.0 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.


In [ ]:
# Téléchargement des données
import os

folder_path = "C:\\Users\\amand\\OneDrive\\Documents\\S9\\IA\\songs\\songs" # Chemin vers le dossier contenant les fichiers audio
dataset = []

for f in os.listdir(folder_path):
    if f.endswith('.flac'):
        path = os.path.join(folder_path, f)
        y, sr = librosa.load(path, sr=None)
        dataset.append({"file": f, "signal": y, "sr": sr})

print(f"Nombre de fichiers chargés : {len(dataset)}")
print("Exemple :", dataset[0]["file"], "durée =", len(dataset[0]["signal"])/dataset[0]["sr"], "s")
data = np.loadtxt('bird_reservoir_data.csv', delimiter=',', skiprows=1)

### Pré-traitement de l'audio

## Classification avec un réservoir

## Interprétation des résultats